# A Taste of Machine Learning

## Introduction

One of the promises of using Notebooks is that we can take advantage of Python's excellent machine learning capabilities.

If you are already a data scientist, this notebook is going to be too trivial for you. We wrote this notebook primarily to expose a data analyst or engineer to Python's machine learning libraries' power.

Also, we'll pick one of the 100's of well-known machine learning algorithms. We decided to expose this algorithm because it is relatively easy to understand.

## K-Means Clustering

K-means clustering is an algorithm that is used to organize data are related based on some criteria. It aims to take a set of observations (or data points) and organize them into clusters (the number of clusters is the `k` in `k-means`).

We'll avoid the mathematical terms here, but you can find some more in-depth discussions at Wikipedia or Wolfram if you are mathematically inclined:

* https://en.wikipedia.org/wiki/K-means_clustering
* https://mathworld.wolfram.com/K-MeansClusteringAlgorithm.html

### Goals

In this story, we will imagine that our boss is asking us if we can figure out where in the US it would be best to place a distribution center for merchandised based on the top us customers. 

We have the top 500 customers in a data file with their addresses. 
Our boss wants us to suggest four geographic locations.

### The Libraries

Before we start, let's introduce the libraries we will use in this notebook:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

We already discussed `pandas`, `matplotlib`, and `numpy` in other notebooks, so let's focus on the new kid, `sklearn`.

`sclearn`, or more complete, `scikit-learn` is a machine learning library written in Python. 

It is a simple and efficient tool that implements many of the popular machine learning algorithms. 
It is built on `numpy` and `matplotlib`, which we have used already. 
It also depends on a scientific library called `scipy`.

We'll only import the `KMeans` algorithm. 

### Evaluate the Dataset of Customers

Let's first look at the dataset our boss gave us. The dataset is in the file `/data/us-500.csv`.

In [ ]:
%%!
head -c2b /data/us-500.csv

Looking at the first five records, we see that we have an address. 

We have a few options. 
1. We could use some geographic lookup API to find GPS
2. We could find some data file that maps zip code to geocoordinates

### Explore Geocoordinates from Zip Codes

We decide to use a simplification of zip codes. 
We'll use the zip to lookup the map coordinates.

We find an open-source dataset that has mapped the zip to longitude and latitude. 
The data is located in the file `/data/zip_lat_long.csv`.


### Read the Data Using Panda

Let's read the data in using panda instead.

#### Reading the People
First, let's read the people.

In [ ]:
people = pd.read_csv('/data/us-500.csv')
people.head()

We realize that we don't need all the columns. 
The only column we really need is the `zip code`.
Let's simply read in that instead

In [ ]:
us = pd.read_csv('/data/zip_lat_long.csv')
us.head()

Let's verify the types to make sure we have the corret types...

In [ ]:
us.dtypes

We see that we have what we want:

* Zip
* Latitude
* Longitude

And for performance, it may be good to use Zip as the index as we know it is unique. 
The `Zip` field is also what we want to use as a lookup.

Pandas always create a unique index and we can use the ZIP as the index, so let's change it a bit

In [ ]:
us = pd.read_csv('/data/zip_lat_long.csv', index_col='zip',usecols=['zip', 'lat', 'long'])
us.head()

In [ ]:
people = pd.read_csv('/data/us-500.csv', usecols=['zip']).astype({'zip': 'int64'})
people.head()

### Merge the Datasets

We have two dataframes, one with the zip code of all the top 500 customers, the second with the geocoordinates of each zip.

Ideally, we want one dataframe that has the zip and geocoordinates together:

In [ ]:
df = people.join(us, on='zip')
df.head()
df[df['long'].isna()]

That is almost it.
We don't need the zip codes now that we have found the geocoordinates, so let's drop the zip from the dataframe. 

We'll simply call this dataset `X`:

In [ ]:
X = df.loc[:,['long', 'lat']]
X.head()

Let's take a look at the data in a scatter diagram:

In [ ]:
X.plot.scatter(x='long', y='lat')

We can start to see the shape of the USA in the points.

There are some outliers that we should probably remove. 
We are only interested in the contiguous states, so we should eliminate observation in Alaska and Hawaii. 
We find those to the very left of the plot. 
It looks like a suitable method to eliminate any observations whose longitude is less than -130.

In [ ]:
X.drop(X[X['long'] < -130].index, inplace=True)
X.plot.scatter(x='long', y='lat');

## K-Means

Let's think about the problem for a second. 

We want to know to minimize the distance from the data center to the customers.
We know the customer's location, so we have to experiment with where we can place the datacenters so that the average distance is as small as possible.

These kinds of problems are what we call NP-complete, meaning, to exhaust all possibilities to find the optimal solution is impractical.

Mathematicians have found ways to approximate the best solutions. One such method is called k-means. 
K-means is a strategy where we run an algorithm to find an approximation of the best way to cluster some data.

The strategy works like this:

1. Collect the observations (the customers with their addresses in our case)
2. Featurize the observations. 
   That is, figure out what's of interest in the dataset and convert them into vectors. 
   For our dataset, the relevant features are the physical address to create a vector using longitude and latitude.
3. Decide the number of clusters we're looking for, called k. 
   In our case, k would be 4 since we're looking for 4 distribution centers.
4. Pick k random values in the dataset and assume this to be the correct clustering points, called centroids.
5. Assign each of the observations to the closest centroid using some measurement of distance. 
   In our case, we can use a so-called euclidian space, that is, we're in a 2-dimensional space where the distance can be measured as an absolute distance using simple euclidian distance. 
   Think of it as a straight line between two map-locations. 
   So, in other words, the distance as the crow flies between the centroid and the customer address.
6. When we have assigned all the observations to a centroid, we recalculate the centroid based to minimize the cluster's distance.
7. Iterate by reassigning the observations to the new centroid.

The animation below shows an example where we have a set of points drawn in a two-dimensional space, and we're trying to find the best 3 cluster points.

<img src='https://upload.wikimedia.org/wikipedia/commons/e/ea/K-means_convergence.gif'/>

## Let's Do It

Enough theory, let's do it.

### Create the Model

First, we need to create the k-means model. 
We are interested in finding 4 locations for our boss, so for us, the number of clusters we are looking for must be set to 4

In [ ]:
model = KMeans(n_clusters=4)

### Fit

Next we need train the model.
This is done with a method called `fit`.

In [ ]:
kmeans = model.fit(X)

### What Are the Clusters?

The model now has calculated the best clusters. 
We can read the result by reading the property called `cluster_centers_`:

In [ ]:
kmeans.cluster_centers_

We have an array with the geocoordinates. Let's make sure that the longitude and latitude are within reasonable ranges. 
They need to be within the range of the original input!

In [ ]:
print('Input Longitude: [(' + str(X['long'].min()) + ')-(' + str(X['long'].max()) + ')]')
print('Input Latitude: [(' + str(X['lat'].min()) + ')-(' + str(X['lat'].max()) + ')]')
print('Cluster Longitude: [(' + str(kmeans.cluster_centers_[0].min()) + ')-(' + str(kmeans.cluster_centers_[0].max()) + ')]')
print('Cluster Latitude: [(' + str(kmeans.cluster_centers_[1].min()) + ')-(' + str(kmeans.cluster_centers_[1].max()) + ')]')
print(kmeans.cluster_centers_[0].min())

## Visualize the Data

### First Attempt

Let's visualize the data by first plotting the customers and then plotting the centroids

In [ ]:
plt.scatter(X['long'], X['lat'])
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1])
plt.show()

Ouch... that is not very easy to read.
That can easily be fixed with a little styling (we'll not go through the options in detail here as we have another notebook where we talk about `matplotlib`).

Let's make the customer locations gray and smaller and use an `X` to mark the centroids.

In [ ]:
plt.scatter(X['long'], X['lat'], color='gray', s=3)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color='red', s=200, marker='X')
plt.show()

## Another Trick

Google Maps allow us to lookup locations using geocoordinates.

We can use the format:

```
http://maps.google.com/?q=<lat>,<lng>
```

So... Let's try to render that using Python as well:

In [ ]:
for c in kmeans.cluster_centers_:
    print("http://maps.google.com/?q=" + str(c[1]) + "," + str(c[0]))

For me, it looks like the recommendations we'll give are:

1. North West of Oklahoma City, OK
2. West of Newark, NJ
3. The third point is in a desolate place. The closest city seems to be Fresno, CA
4. The fourth point is also in the middle of nowhere. The best recommendation seems to be somewhere in the triangle formed by Knoxville (TN), Nashville (TN), and Loisville (KY)



## Reflection

In this notebook, we've exposed you to some machine learning. 
We used an algorithm called k-means clustering. 

It is one of the easier to understand algorithms, but it shows that even the simpler algorithms have teeth.
It is also worth mentioning that we've only scratched the surface of what the SciKit Learning library can do. 
We've also only scratched the surface of what can be done with clustering and k-means.

For example, in k-means, there is a complete theory around various aspects of the algorithm.
For example:

* **How do you pick the initial centroids?** 
  SciKit Learn, by default, uses an algorithm called `KMeans++`, but it supports other methods as well.
  The selection of algorithm can make the model work differently (mostly, it is a performance issue).
* **How do you measure distance?** 
  We used the default method which is to use a sum of square distances in a euclidian space, but there are all ways to measure distance as well. 
* **What algorithm to use?**
  SciKit Learn uses Elkan’s algorithm, but that is also configurable.
* **How do I choose `k`?**
  In our case, we knew that `k` should be 4. 
  In other cases, we may not know what `k` should be.
  There are at least 10 known methods to resolve `k`.
  Here is a link to a good article showing 3 of the methods (https://www.datanovia.com/en/lessons/determining-the-optimal-number-of-clusters-3-must-know-methods/).

Add to that, SciKit Learn provides a number of clustering algorithms:

* **Affinity propagation**. 
  Works really well with smaller number of samples and with many (even uneven) clusters. 
  Also good for non-flat geometry.
* **Mean-shift**
  The use cases are similar to that of the `Affinity Propagation`, but usually used where there are fewer clusters.
* **Spectral clustering**.
  Handles larger samples than `Mean-Shift` and `Affinity Propagation` but handles larger sample sets. 
  Typically used with feweer clusters but works well for non-flat geometries.
* **Ward hierarchical clustering**.
  Can handle large sample sets and many clusters and allows you to specify connectivity constraints.
* **Agglomerative clustering**.
  Handles large number of samples and clusters.
  Can use non-euclidean distances as well as connectivity constraints.
* **DBSCAN**.
  Can hadle very large sample sets.
  Supports non-flat geometry and uneven cluster sizes.
* **OPTICS**
  Supports very large sample sets and large clusters.
  Supports non-flat geometry, uneven cluster sizes, and variable cluster density.
* **Gaussian mixtures**.
  Not very scalable. 
  Good for density estimations on flat geometry. 
  Uses Mahalanobis distances to  centers.
* **Birch**.
  Supports large clusters and sample sets.
  Limited to euclidean distances.

I'm not trying to scare anyone away from machine learning.
My point only, to become a data scientist requires a commitment to relatively abstract concepts. 

The good news is that libraries like SciKit Learn are making machine learning accessible to everyone.
One of their goals is to allow you to use some of the algorithms without completely understanding the underlying theory.